In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import pandas as pd
from openai import OpenAI

In [2]:
load_dotenv(find_dotenv(), override=True)

True

In [3]:
CSV_CONTEXTS = "05.3_selected_contexts.csv"
CSV_QUESTIONS = "05.0_questions.csv"

In [4]:
def get_llm_client(llm_choice):
    if llm_choice == "GROQ":
        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )
        return client
    elif llm_choice == "OPENAI":
        load_dotenv()  # load environment variables from .env fil
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        return client
    else:
        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [5]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY exists and begins sk-proj-vIt-L1...
GROQ_API_KEY exists and begins gsk_0yKDCuUXkz...
LLM_CHOICE: GROQ - MODEL: llama-3.3-70b-versatile


In [6]:
list_questions = []

with open(CSV_QUESTIONS, "r") as f:
    for line in f:
        list_questions.append(line.strip())

In [7]:
print(list_questions)

['What was the climate change about last year?', 'Why was food price volatility so high?', 'What does CRISPR-Cas9 enable?', 'Optimise self driving?']


In [8]:
df_contexts = pd.read_csv(CSV_CONTEXTS, sep="|")
df_contexts.head()

,id,question,contexts,metadata
0,2267ceda-69dd-4aab-b2cb-8c4e302dfe4c,What was the climate change about last year?,Climate change poses significant risks to glob...,"{'author': 'Maria Rodriguez', 'keywords_str': ..."
1,2267ceda-69dd-4aab-b2cb-8c4e302dfe4c,Why was food price volatility so high?,Climate change poses significant risks to glob...,"{'keywords_str': 'climate change, food securit..."
2,ac412bdd-1013-4e89-b6c4-0cdca7c0edfb,What does CRISPR-Cas9 enable?,Genetic editing techniques show promise for tr...,"{'year': 2024, 'source': 'biotech_research', '..."
3,af9586c4-eba3-42a0-b21c-1fbc9ec83f6d,Optimise self driving?,Urban planning strategies are adapting to acco...,"{'keywords_str': 'urban planning, autonomous v..."


In [10]:
list_answers = []

for i, question in enumerate(list_questions):
    context = df_contexts.loc[i, "contexts"]
    # print(i, question, context)
    prompts = [
        {
            "role": "system",
            "content": f"You are a RAG specialist. You are given a question and a context. You will answer the question based on the context. The answer should be a short and concise answer: {context}",
        },
        {"role": "user", "content": question},
    ]
    # get llm answer
    response = client.chat.completions.create(
        model=MODEL, messages=prompts, temperature=0.7
    )
    # print(response.choices[0].message.content)
    answer = response.choices[0].message.content
    list_answers.append(answer)
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f"Context: {context}")
    print("-" * 100)

Question: What was the climate change about last year?
Answer: The context doesn't provide information about climate change for a specific year, "last year". It discusses the general impacts of climate change on global food security and agricultural systems.
Context: Climate change poses significant risks to global food security and agricultural systems, threatening the livelihoods of billions of people worldwide and challenging the fundamental stability of food production networks. Rising temperatures, altered precipitation patterns, and increased frequency of extreme weather events are disrupting traditional farming practices and crop yields across different geographic regions. Agricultural communities face unprecedented challenges as changing climate conditions affect soil quality, water availability, and pest populations, forcing farmers to adapt their cultivation methods and crop selections. Developing nations are particularly vulnerable to these impacts, as their agricultural sec

In [ ]:
# convert list_answers to pandas dataframe
df_answers = pd.DataFrame({"answer": list_answers})
df_answers.to_csv("05.5_answers.csv", index=False)
df_answers

,answer
0,The context doesn't provide information about ...
1,Food price volatility was high due to climate-...
2,CRISPR-Cas9 enables precise modifications to D...
3,"To optimize self-driving technology, city plan..."


In [15]:
# merge df_contexts and df_answers
df_merged = pd.concat([df_contexts, df_answers], axis=1)
df_merged

,id,question,contexts,metadata,answer
0,2267ceda-69dd-4aab-b2cb-8c4e302dfe4c,What was the climate change about last year?,Climate change poses significant risks to glob...,"{'author': 'Maria Rodriguez', 'keywords_str': ...",The context doesn't provide information about ...
1,2267ceda-69dd-4aab-b2cb-8c4e302dfe4c,Why was food price volatility so high?,Climate change poses significant risks to glob...,"{'keywords_str': 'climate change, food securit...",Food price volatility was high due to climate-...
2,ac412bdd-1013-4e89-b6c4-0cdca7c0edfb,What does CRISPR-Cas9 enable?,Genetic editing techniques show promise for tr...,"{'year': 2024, 'source': 'biotech_research', '...",CRISPR-Cas9 enables precise modifications to D...
3,af9586c4-eba3-42a0-b21c-1fbc9ec83f6d,Optimise self driving?,Urban planning strategies are adapting to acco...,"{'keywords_str': 'urban planning, autonomous v...","To optimize self-driving technology, city plan..."


In [17]:
# save df_merged to csv
df_merged.to_csv("05.6_evals_dataset.csv", index=False)

### We can now perform the evaluations